In [371]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import joblib

from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import spacy
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize

import statistics


import re
from os import path
from PIL import Image
import plotly
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.collocations import *
nltk.download('averaged_perceptron_tagger')

from textblob import TextBlob

import seaborn as sns

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hasee\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [372]:
def normalize_list(data):
	x_min = min(data)
	x_max = max(data)
	if x_max-x_min == 0:
		return None
	return [(x-x_min)/(x_max-x_min) for x in data]

In [373]:
df = pd.read_csv('../../data_with_senti/ge_sentiments.csv', encoding='utf-8')

In [374]:
df.head()

,Unnamed: 0,Company,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader
0,0,GE,5.0,0,NaN,Good company,2022-02-25,Technical Product Manager,"Detroit, MI",2,0,1,Competitive salary and training opportunities,Extreme work load and unclear responsibility.,NaN,0.44,Competitive salary and training opportunitiesE...,-0.08
1,1,GE,4.0,0,NaN,Its electric,2022-02-24,Anonymous Employee,NaN,0,0,0,"lots of power, lots of light","to bright sometimes, less power",NaN,0.00,"lots of power, lots of lightto bright sometime...",0.44
2,2,GE,5.0,1,less than 1 year,Good Company to start with,2022-02-24,Lead Engineer,"Harlem, FL",-1,1,-1,"Everything, you've s job, be happy","Nothing, actually. Everything is fine.",NaN,0.44,"Everything, you've s job, be happyNothing, act...",0.20
3,3,GE,4.0,0,more than 5 years,great experience,2022-02-23,Manager Total Rewards,"Los Angeles, CA",0,2,0,"loved the team, the flexibility to choose spec...",does not offer stock as part of the comp packa...,NaN,0.62,"loved the team, the flexibility to choose spec...",0.84
4,4,GE,4.0,1,more than 5 years,Great place for experience,2022-02-21,Gas Turbine Technician,"Houston, TX",0,0,0,Job security and lots of OT,Raises aren’t good and management expects loyalty,NaN,0.62,Job security and lots of OTRaises aren’t good ...,0.83


In [375]:
df.columns

Index(['Unnamed: 0', 'Company', 'Star_Rating', 'Current_Employee', 'Work_Duration', 'Review_Title', 'Date_Posted', 'Job_Title', 'Job_Location', 'Recommend', 'CEO_Approval', 'Business_Outlook', 'Pros', 'Cons', 'Advice_to_Management', 'Review_Title_sentiment_score_vader', 'Pros_And_Cons', 'Pros_And_Cons_sentiment_score_vader'], dtype='object')

In [376]:
df.drop(['Unnamed: 0', 'Company'], axis=1, inplace=True)

In [377]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader
0,5.0,0,NaN,Good company,2022-02-25,Technical Product Manager,"Detroit, MI",2,0,1,Competitive salary and training opportunities,Extreme work load and unclear responsibility.,NaN,0.44,Competitive salary and training opportunitiesE...,-0.08
1,4.0,0,NaN,Its electric,2022-02-24,Anonymous Employee,NaN,0,0,0,"lots of power, lots of light","to bright sometimes, less power",NaN,0.00,"lots of power, lots of lightto bright sometime...",0.44
2,5.0,1,less than 1 year,Good Company to start with,2022-02-24,Lead Engineer,"Harlem, FL",-1,1,-1,"Everything, you've s job, be happy","Nothing, actually. Everything is fine.",NaN,0.44,"Everything, you've s job, be happyNothing, act...",0.20
3,4.0,0,more than 5 years,great experience,2022-02-23,Manager Total Rewards,"Los Angeles, CA",0,2,0,"loved the team, the flexibility to choose spec...",does not offer stock as part of the comp packa...,NaN,0.62,"loved the team, the flexibility to choose spec...",0.84
4,4.0,1,more than 5 years,Great place for experience,2022-02-21,Gas Turbine Technician,"Houston, TX",0,0,0,Job security and lots of OT,Raises aren’t good and management expects loyalty,NaN,0.62,Job security and lots of OTRaises aren’t good ...,0.83


In [378]:
df['Date_Posted'] = pd.to_datetime(df['Date_Posted'], dayfirst=True)

In [379]:
quarters = pd.Series(pd.date_range('2012-01-01', '2022-12-31', freq='QS'))

In [380]:
for i, row in df.iterrows():
	if isinstance(row['Work_Duration'], str):
		if 'less' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0] - 1
			# row['Work_Duration'] = years[0] - 1
		elif 'more' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0]
			# row['Work_Duration'] = years[0]

In [381]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader
0,5.0,0,NaN,Good company,2022-02-25,Technical Product Manager,"Detroit, MI",2,0,1,Competitive salary and training opportunities,Extreme work load and unclear responsibility.,NaN,0.44,Competitive salary and training opportunitiesE...,-0.08
1,4.0,0,NaN,Its electric,2022-02-24,Anonymous Employee,NaN,0,0,0,"lots of power, lots of light","to bright sometimes, less power",NaN,0.00,"lots of power, lots of lightto bright sometime...",0.44
2,5.0,1,0,Good Company to start with,2022-02-24,Lead Engineer,"Harlem, FL",-1,1,-1,"Everything, you've s job, be happy","Nothing, actually. Everything is fine.",NaN,0.44,"Everything, you've s job, be happyNothing, act...",0.20
3,4.0,0,5,great experience,2022-02-23,Manager Total Rewards,"Los Angeles, CA",0,2,0,"loved the team, the flexibility to choose spec...",does not offer stock as part of the comp packa...,NaN,0.62,"loved the team, the flexibility to choose spec...",0.84
4,4.0,1,5,Great place for experience,2022-02-21,Gas Turbine Technician,"Houston, TX",0,0,0,Job security and lots of OT,Raises aren’t good and management expects loyalty,NaN,0.62,Job security and lots of OTRaises aren’t good ...,0.83


In [382]:
df.drop('Job_Title', axis=1, inplace=True)

In [383]:
df.drop('Advice_to_Management', axis=1, inplace=True)

In [384]:
df.drop('Job_Location', axis=1, inplace=True)

In [385]:
df['Work_Duration'].fillna(df['Work_Duration'].mean(),inplace=True)

# other value aggregates

In [386]:
star_rating_aggregates = dict()
title_senti_aggregates = dict()
pnc_senti_aggregates = dict()
work_duration_aggregates = dict()

current_employee_aggregates = dict()
recommend_aggregates = dict()
ceo_approval_aggregates = dict()
business_outlook_aggregates = dict()



# percentages
recommend_percentages = dict()
ceo_percentages = dict()
business_percentages = dict()
counts = dict()
count = 0

In [387]:

for q in quarters:
	star_rating_aggregates[f'Q{count+1}'] = 0
	title_senti_aggregates[f'Q{count+1}'] = 0
	pnc_senti_aggregates[f'Q{count+1}'] = 0
	work_duration_aggregates[f'Q{count+1}'] = 0


	current_employee_aggregates[f'Q{count+1}'] = []
	recommend_aggregates[f'Q{count+1}'] = []
	ceo_approval_aggregates[f'Q{count+1}'] = []
	business_outlook_aggregates[f'Q{count+1}'] = []

	recommend_percentages[f'Q{count+1}'] = dict()
	ceo_percentages[f'Q{count+1}'] = dict()
	business_percentages[f'Q{count+1}'] = dict()
	for i in range(-1, 3):
		recommend_percentages[f'Q{count+1}'][i] = 0 
		ceo_percentages[f'Q{count+1}'][i] = 0
		business_percentages[f'Q{count+1}'][i] = 0


	counts[f'Q{count+1}'] = 0
	count += 1

for idx, row in df.iterrows():
	for i in range(len(quarters)):
		if i < len(quarters) - 1:
			if row['Date_Posted'] >= quarters[i] and row['Date_Posted'] < quarters[i+1]:
				star_rating_aggregates[f'Q{i+1}'] += row['Star_Rating']
				title_senti_aggregates[f'Q{i+1}'] += row['Review_Title_sentiment_score_vader']
				pnc_senti_aggregates[f'Q{i+1}'] += row['Pros_And_Cons_sentiment_score_vader']
				work_duration_aggregates[f'Q{i+1}'] += row['Work_Duration']


				current_employee_aggregates[f'Q{i+1}'].append(row['Current_Employee'])
				recommend_aggregates[f'Q{i+1}'].append(row['Recommend'])
				ceo_approval_aggregates[f'Q{i+1}'].append(row['CEO_Approval'])
				business_outlook_aggregates[f'Q{i+1}'].append(row['Business_Outlook'])

				recommend_percentages[f'Q{i+1}'][row['Recommend']] += 1
				ceo_percentages[f'Q{i+1}'][row['CEO_Approval']] += 1
				business_percentages[f'Q{i+1}'][row['Business_Outlook']] += 1
				
				counts[f'Q{i+1}'] += 1

In [388]:

avg_star_ratings = []
avg_title_senti = []
avg_pnc_senti = []
avg_work_duration = []

avg_current_employee = []
avg_recommend = []
avg_ceo_approval = []
avg_business_outlook = []

# percentages 

recommend_minus_percentages = []
ceo_minus_percentages = []
business_minus_percentages = []

recommend_unknown_percentages = []
ceo_unknown_percentages = []
business_unknown_percentages = []

recommend_same_percentages = []
ceo_same_percentages = []
business_same_percentages = []

recommend_plus_percentages = []
ceo_plus_percentages = []
business_plus_percentages = []


quarts = []


In [389]:
for quarter, value in star_rating_aggregates.items():

	if counts[quarter] != 0:

		avg_star_ratings.append(star_rating_aggregates[quarter] / counts[quarter])
		avg_title_senti.append(title_senti_aggregates[quarter] / counts[quarter])
		avg_pnc_senti.append(pnc_senti_aggregates[quarter] / counts[quarter])
		avg_work_duration.append(work_duration_aggregates[quarter] / counts[quarter])


		avg_current_employee.append(statistics.mode(current_employee_aggregates[quarter]))
		avg_recommend.append(statistics.mode(recommend_aggregates[quarter]))
		avg_ceo_approval.append(statistics.mode(ceo_approval_aggregates[quarter]))
		avg_business_outlook.append(statistics.mode(business_outlook_aggregates[quarter]))

		recommend_minus_percentages.append(recommend_percentages[quarter][-1] / counts[quarter])
		ceo_minus_percentages.append(ceo_percentages[quarter][-1] / counts[quarter])
		business_minus_percentages.append(business_percentages[quarter][-1] / counts[quarter])

		recommend_unknown_percentages.append(recommend_percentages[quarter][0] / counts[quarter])
		ceo_unknown_percentages.append(ceo_percentages[quarter][0] / counts[quarter])
		business_unknown_percentages.append(business_percentages[quarter][0] / counts[quarter])

		recommend_same_percentages.append(recommend_percentages[quarter][1] / counts[quarter])
		ceo_same_percentages.append(ceo_percentages[quarter][1] / counts[quarter])
		business_same_percentages.append(business_percentages[quarter][1] / counts[quarter])

		recommend_plus_percentages.append(recommend_percentages[quarter][2] / counts[quarter])
		ceo_plus_percentages.append(ceo_percentages[quarter][2] / counts[quarter])
		business_plus_percentages.append(business_percentages[quarter][2] / counts[quarter])
		
		quarts.append(quarter)

<hr>

In [390]:
def clean_text(text):
	text=re.sub(r"\?", ".", text)
	text=re.sub(r"\!", ".", text)
	text=re.sub(r'([.])\1+', r'\1', text)
	rexp=r"\.(?=[A-Za-z]{1,15})"
	text=re.sub(rexp, ". ", text)
	return text

#this function extracts lemmas of nouns, verbs and adjectives
def lemma_words_pos_filtered(text):
    word_list=[]
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    pos_tokens=nltk.pos_tag(tokens)
    for pos in pos_tokens:
        if (pos[1].startswith('N')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.NOUN)]
        if (pos[1].startswith('V')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.VERB)]
        if (pos[1].startswith('J')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.ADJ)]
    word_list=[word for word in word_list if word not in  
                                    stopwords.words('english') ]
    return  " ".join(word_list)

#to keep only unique tuples
def unique_tuples(list_of_tuples):
     list_ = [tuple(sorted(t)) for t in list_of_tuples]
     list_ = list(set(list_))
     return list_

def findbigramsintext(text):
	mylist=nltk.word_tokenize(text)
	list1=[x for x in mylist]
	feature_list = []
	for i in range(length_features2):
	# In each iteration, add an empty list to the main list      
			feature_list.append([])
	i=0
	# bigram_features is the list of unique set of bigrams found after collocation
	for l in bigram_features:
			list2=[x for x in l]
			result =  all(elem in list1  for elem in list2)
			if result: 
				feature_list[i].append(' '.join(list2))
			i=i+1
	return feature_list


def keepnonempty(list1):
       mylist= [x for x in list1 if x != []]
       return mylist
def flatten_list(row_list):
        l = [item for inner_list in row_list for item in inner_list]
        return l


# text cleaning

In [391]:

df['Cleaned_Pros_and_Cons_Text'] = df['Pros_And_Cons'].apply(sent_tokenize)

In [392]:
len(df)

11873

In [393]:
df = df.explode('Cleaned_Pros_and_Cons_Text', ignore_index=False)

In [394]:
len(df)

26024

In [395]:
df['lemma_words_pos_filtered'] = df['Cleaned_Pros_and_Cons_Text'].apply(lemma_words_pos_filtered)

In [396]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words( " ".join(df['lemma_words_pos_filtered']).split(),window_size=5)
#keeping bigrams that occur at least 5 times
finder.apply_freq_filter(5)
bigram_list_tuples=finder.nbest(bigram_measures.likelihood_ratio, 20)

In [397]:
bigram_list_tuples

[('life', 'balance'),
 ('work', 'life'),
 ('none', 'none'),
 ('work', 'balance'),
 ('red', 'tape'),
 ('work-life', 'balance'),
 ('work/life', 'balance'),
 ('long', 'term'),
 ('leadership', 'program'),
 ('long', 'hour'),
 ('great', 'place'),
 ('job', 'security'),
 ('smart', 'people'),
 ('large', 'company'),
 ('n', 'n'),
 ('decision', 'making'),
 ('short', 'term'),
 ('upper', 'management'),
 ('work', 'environment'),
 ('flexible', 'hour')]

In [398]:
topic_polarities

{'balance life': [0.022103174603174607,
  0.43514209133945975,
  0.40095598845598845,
  0.09787698412698412,
  0.6475,
  -0.09285714285714285,
  0.2955371900826446,
  0.12892405459554218,
  0.35748602426380205,
  0.44780158730158726,
  0.19769095102428436,
  0.10663690476190478,
  0.3624,
  0.3908040438397581,
  0.3130864340685769,
  0.3710273654916512,
  0.39832227891156463,
  0.35162509448223733,
  0.23882731637361268,
  0.1282589285714286,
  0.27593537414965985,
  0.19452748383303942,
  0.34542200854700855,
  0.03868452380952381,
  0.3173665490887713,
  0.2774743609565038,
  0.40675015460729746,
  0.3201851851851852,
  0.4339502164502164,
  0.3118492063492063,
  0.16947128851540616,
  0.3807391534391534,
  0.2673076923076923,
  0.30598232323232327,
  0.3058300395256917,
  0.3010662427849927,
  0.37551711389477344,
  0.2840090944720574,
  0.18934794372294372,
  0.28204331419609197,
  0.24457199546485262],
 'balance work': [0.06411706349206349,
  0.4015072983384387,
  0.38657395382395

In [399]:
bigram_features=unique_tuples(bigram_list_tuples)
length_features2=len(bigram_features)

In [400]:
df['bigram_list'] = df['lemma_words_pos_filtered'].apply(findbigramsintext)

In [401]:
df['bigrams']=df['bigram_list'].apply(keepnonempty)
df['bigrams']=df['bigrams'].apply(flatten_list)

In [402]:
#Sentiment scores are calculated using libraries TextBlob and Vader
df['polarity'] = df['Cleaned_Pros_and_Cons_Text'].apply(lambda x: TextBlob(x).sentiment[0])

In [403]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader,Cleaned_Pros_and_Cons_Text,lemma_words_pos_filtered,bigram_list,bigrams,polarity
0,5.0,0,4.414544,Good company,2022-02-25,2,0,1,Competitive salary and training opportunities,Extreme work load and unclear responsibility.,0.44,Competitive salary and training opportunitiesE...,-0.08,Competitive salary and training opportunitiesE...,competitive salary training opportunitiesextre...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.000000
1,4.0,0,4.414544,Its electric,2022-02-24,0,0,0,"lots of power, lots of light","to bright sometimes, less power",0.00,"lots of power, lots of lightto bright sometime...",0.44,"lots of power, lots of lightto bright sometime...",lot power lot lightto bright less power,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.266667
2,5.0,1,0.000000,Good Company to start with,2022-02-24,-1,1,-1,"Everything, you've s job, be happy","Nothing, actually. Everything is fine.",0.44,"Everything, you've s job, be happyNothing, act...",0.20,"Everything, you've s job, be happyNothing, act...",everything 've job happynothing,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.000000
2,5.0,1,0.000000,Good Company to start with,2022-02-24,-1,1,-1,"Everything, you've s job, be happy","Nothing, actually. Everything is fine.",0.44,"Everything, you've s job, be happyNothing, act...",0.20,Everything is fine.,everything fine,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.416667
3,4.0,0,5.000000,great experience,2022-02-23,0,2,0,"loved the team, the flexibility to choose spec...",does not offer stock as part of the comp packa...,0.62,"loved the team, the flexibility to choose spec...",0.84,"loved the team, the flexibility to choose spec...",love team flexibility choose special projectsd...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.528571


In [404]:
copy_df = df.copy()

In [405]:
copy_df = copy_df.explode('bigrams', ignore_index=False)

In [406]:
res = copy_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

In [407]:
print(res)

               bigrams  polarity
0         balance life  0.282088
1         balance work  0.282547
2    balance work-life  0.298795
3    balance work/life  0.154482
4        company large  0.199464
5      decision making  0.142553
6     environment work  0.313339
7        flexible hour  0.215605
8          great place  0.538977
9            hour long  0.102862
10        job security  0.227693
11  leadership program  0.274696
12           life work  0.294257
13           long term  0.113635
14    management upper  0.097152
15                 n n  0.282992
16           none none  0.147754
17        people smart  0.269785
18            red tape  0.136481
19          short term  0.109535


In [408]:
all_bigrams = res['bigrams'].tolist()
print(all_bigrams)

['balance life', 'balance work', 'balance work-life', 'balance work/life', 'company large', 'decision making', 'environment work', 'flexible hour', 'great place', 'hour long', 'job security', 'leadership program', 'life work', 'long term', 'management upper', 'n n', 'none none', 'people smart', 'red tape', 'short term']


In [409]:
result = copy_df[['bigrams', 'polarity']].groupby('bigrams')['polarity'].mean()

In [410]:
topic_polarities = dict()
for b in all_bigrams:
	topic_polarities[b] = []

In [411]:
polarity_results = []
for i in range(len(quarters)):
	if i < len(quarters) - 1:
		qtr_start = quarters[i]
		qtr_end = quarters[i+1]
		cur_df = df[(df['Date_Posted'] >= qtr_start) & (df['Date_Posted'] < qtr_end)]
		if(len(cur_df) == 0):
			continue
		cur_df = cur_df.explode('bigrams', ignore_index=False)
		results = cur_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

		for bigram in all_bigrams:
			if bigram in results['bigrams'].tolist():
				polarity = results[results['bigrams'] == bigram]['polarity'].tolist()[0]
				topic_polarities[bigram].append(polarity)
				# polarity_results.append(results[results['bigrams'] == bigram]['polarity'].tolist()[0])
			else:
				topic_polarities[bigram].append(None)

		# for i, row in results.iterrows():
		# 	if row['bigrams'] in topic_polarities:
		# 		topic_polarities[row['bigrams']].append(row['polarity'])
		# 	else:
		# 		topic_polarities[row['bigrams']] = [row['polarity']]
		polarity_results.append(results)


In [412]:
len(topic_polarities)

20

In [413]:
len(polarity_results)

41

In [414]:
for key, value in topic_polarities.items():
	print(key, len(value))

balance life 41
balance work 41
balance work-life 41
balance work/life 41
company large 41
decision making 41
environment work 41
flexible hour 41
great place 41
hour long 41
job security 41
leadership program 41
life work 41
long term 41
management upper 41
n n 41
none none 41
people smart 41
red tape 41
short term 41


In [415]:
print(counts)

{'Q1': 81, 'Q2': 78, 'Q3': 121, 'Q4': 128, 'Q5': 166, 'Q6': 131, 'Q7': 151, 'Q8': 177, 'Q9': 230, 'Q10': 241, 'Q11': 252, 'Q12': 206, 'Q13': 259, 'Q14': 344, 'Q15': 431, 'Q16': 357, 'Q17': 422, 'Q18': 398, 'Q19': 413, 'Q20': 392, 'Q21': 466, 'Q22': 422, 'Q23': 352, 'Q24': 351, 'Q25': 281, 'Q26': 217, 'Q27': 188, 'Q28': 148, 'Q29': 148, 'Q30': 177, 'Q31': 250, 'Q32': 194, 'Q33': 171, 'Q34': 160, 'Q35': 252, 'Q36': 348, 'Q37': 571, 'Q38': 411, 'Q39': 365, 'Q40': 336, 'Q41': 257, 'Q42': 0, 'Q43': 0, 'Q44': 0}


In [456]:
final = pd.DataFrame(
{   'quarters': quarts,
	'ratings': normalize_list(avg_star_ratings),
	'title_senti': normalize_list(avg_title_senti),
	'pnc_senti': normalize_list(avg_pnc_senti),
	'work_duration': normalize_list(avg_work_duration),
	'current_employee': avg_current_employee,
	'recommend': avg_recommend,
	'ceo_approval': avg_ceo_approval,
	'business_outlook': avg_business_outlook,
	'recommend_minus': normalize_list(recommend_minus_percentages),
	'ceo_minus': normalize_list(ceo_minus_percentages),
	'business_minus': normalize_list(business_minus_percentages),
	'recommend_unknown': normalize_list(recommend_unknown_percentages),
	'ceo_unknown': normalize_list(ceo_unknown_percentages),
	'business_unknown': normalize_list(business_unknown_percentages),
	'recommend_same': normalize_list(recommend_same_percentages),
	'ceo_same': normalize_list(ceo_same_percentages),
	'business_same': normalize_list(business_same_percentages),
	'recommend_plus': normalize_list(recommend_plus_percentages),
	'ceo_plus': normalize_list(ceo_plus_percentages),
	'business_plus': normalize_list(business_plus_percentages),
})

In [457]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,0.463219,0.000000,0.340773,0.000000,1.000000,None,0.814983,0.000000,0.728025,1.000000,0.000000
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.402083,0.130598,0.612179,None,0.607133,0.341880,0.564629,0.924384,0.335772
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.064420,0.183721,0.000000,None,0.568201,0.991736,0.762177,0.751299,0.715944
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.000000,0.170772,0.000150,None,0.278147,1.000000,0.870440,0.945994,0.645312
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.043089,0.179941,0.039794,None,0.483256,0.867470,1.000000,0.933905,0.849545
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,0.605006,0.433133,0.105704,0.209387,0.023048,None,0.780053,0.916031,0.677906,0.600114,0.676671
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,0.231376,0.554699,0.094132,0.241033,0.021460,None,0.675224,0.830022,0.874324,0.766059,0.653755
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,1.000000,0.412158,0.122789,0.181647,0.060028,None,0.659204,0.828625,0.663875,0.600668,0.694309
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,0.316219,0.270192,0.311790,0.233444,0.043938,None,0.578993,0.892754,0.623916,0.825595,0.779576
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,0.496960,0.313916,0.065828,0.238199,0.034472,None,0.491722,0.840941,0.913437,0.820266,0.802510


In [458]:
all_bigrams

['balance life',
 'balance work',
 'balance work-life',
 'balance work/life',
 'company large',
 'decision making',
 'environment work',
 'flexible hour',
 'great place',
 'hour long',
 'job security',
 'leadership program',
 'life work',
 'long term',
 'management upper',
 'n n',
 'none none',
 'people smart',
 'red tape',
 'short term']

In [459]:
for key,value in topic_polarities.items():
	col_name = key.replace(' ', '_')
	final[col_name] = value

In [460]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,0.463219,0.000000,0.340773,0.000000,1.000000,None,0.814983,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,NaN,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,NaN,0.000000,NaN,0.192976,0.093333,0.108929
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.402083,0.130598,0.612179,None,0.607133,0.341880,0.564629,0.924384,0.335772,0.435142,0.401507,0.377778,NaN,0.176346,NaN,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,NaN,NaN,0.508306,0.016667,0.078571
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.064420,0.183721,0.000000,None,0.568201,0.991736,0.762177,0.751299,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,NaN,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,NaN,-1.000000,0.330586,0.062500,NaN
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.000000,0.170772,0.000150,None,0.278147,1.000000,0.870440,0.945994,0.645312,0.097877,0.097877,NaN,NaN,0.179857,NaN,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,NaN,-0.110000,0.439596,NaN,NaN
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.043089,0.179941,0.039794,None,0.483256,0.867470,1.000000,0.933905,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,NaN,0.229449,0.145833,0.447823,0.250556,NaN,0.227183,0.533160,0.483333,-0.009259,NaN,NaN,0.303163,0.189983,0.000000
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,0.605006,0.433133,0.105704,0.209387,0.023048,None,0.780053,0.916031,0.677906,0.600114,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,NaN,0.317708,0.136364,-0.254762,0.000000,NaN,NaN,0.447619,0.231664,NaN
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,0.231376,0.554699,0.094132,0.241033,0.021460,None,0.675224,0.830022,0.874324,0.766059,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,NaN,NaN,0.182041,0.300379,0.122743,0.051042
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,1.000000,0.412158,0.122789,0.181647,0.060028,None,0.659204,0.828625,0.663875,0.600668,0.694309,0.128924,0.128924,NaN,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,NaN,0.115000,0.373214,0.048258,NaN
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,0.316219,0.270192,0.311790,0.233444,0.043938,None,0.578993,0.892754,0.623916,0.825595,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,NaN,0.109722,0.491667,0.288232,0.212875,0.267262,NaN,0.200000,0.355685,0.099582,0.144286
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,0.496960,0.313916,0.065828,0.238199,0.034472,None,0.491722,0.840941,0.913437,0.820266,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,NaN,0.193228,0.413597,0.101927,0.143750,NaN,0.168287,0.249233,0.297321,0.080000


In [461]:
for key, value in topic_polarities.items():
	col = key.replace(' ', '_')
	final[col].fillna(final[col].mean(),inplace=True)

In [462]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,0.463219,0.000000,0.340773,0.000000,1.000000,None,0.814983,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,0.216641,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,0.094461,0.000000,0.129221,0.192976,0.093333,0.108929
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.402083,0.130598,0.612179,None,0.607133,0.341880,0.564629,0.924384,0.335772,0.435142,0.401507,0.377778,0.210688,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.129221,0.508306,0.016667,0.078571
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.064420,0.183721,0.000000,None,0.568201,0.991736,0.762177,0.751299,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.216641,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.112216
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.000000,0.170772,0.000150,None,0.278147,1.000000,0.870440,0.945994,0.645312,0.097877,0.097877,0.330022,0.210688,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.166832,0.112216
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.043089,0.179941,0.039794,None,0.483256,0.867470,1.000000,0.933905,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.204370,0.227183,0.533160,0.483333,-0.009259,0.286658,0.129221,0.303163,0.189983,0.000000
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,0.605006,0.433133,0.105704,0.209387,0.023048,None,0.780053,0.916031,0.677906,0.600114,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.204370,0.317708,0.136364,-0.254762,0.000000,0.286658,0.129221,0.447619,0.231664,0.112216
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,0.231376,0.554699,0.094132,0.241033,0.021460,None,0.675224,0.830022,0.874324,0.766059,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.094461,0.286658,0.182041,0.300379,0.122743,0.051042
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,1.000000,0.412158,0.122789,0.181647,0.060028,None,0.659204,0.828625,0.663875,0.600668,0.694309,0.128924,0.128924,0.330022,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.112216
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,0.316219,0.270192,0.311790,0.233444,0.043938,None,0.578993,0.892754,0.623916,0.825595,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.107423,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,0.496960,0.313916,0.065828,0.238199,0.034472,None,0.491722,0.840941,0.913437,0.820266,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.204370,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000


In [463]:
earnings = [
-1042,
-3843,
1257,
-1131,
-2802,
2636,
-1144,
-1987,
6199,
728,
-9423,
127,
3588,
761,
-22770,
800,
-1147,
-10820,
1360,
1057,
-83,
2336,
2027,
2908,
228,
6301,
2506,
-1360,
-13570,
5152,
3537,
3545,
2999,
3206,
3191,
3133,
3527,
4011,
3491,
3105,
3034
]

earnings = earnings[::-1]

In [464]:
final['earnings'] = earnings

In [465]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term,earnings
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,0.463219,0.000000,0.340773,0.000000,1.000000,None,0.814983,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,0.216641,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,0.094461,0.000000,0.129221,0.192976,0.093333,0.108929,3034
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.402083,0.130598,0.612179,None,0.607133,0.341880,0.564629,0.924384,0.335772,0.435142,0.401507,0.377778,0.210688,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.129221,0.508306,0.016667,0.078571,3105
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.064420,0.183721,0.000000,None,0.568201,0.991736,0.762177,0.751299,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.216641,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.112216,3491
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.000000,0.170772,0.000150,None,0.278147,1.000000,0.870440,0.945994,0.645312,0.097877,0.097877,0.330022,0.210688,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.166832,0.112216,4011
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.043089,0.179941,0.039794,None,0.483256,0.867470,1.000000,0.933905,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.204370,0.227183,0.533160,0.483333,-0.009259,0.286658,0.129221,0.303163,0.189983,0.000000,3527
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,0.605006,0.433133,0.105704,0.209387,0.023048,None,0.780053,0.916031,0.677906,0.600114,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.204370,0.317708,0.136364,-0.254762,0.000000,0.286658,0.129221,0.447619,0.231664,0.112216,3133
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,0.231376,0.554699,0.094132,0.241033,0.021460,None,0.675224,0.830022,0.874324,0.766059,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.094461,0.286658,0.182041,0.300379,0.122743,0.051042,3191
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,1.000000,0.412158,0.122789,0.181647,0.060028,None,0.659204,0.828625,0.663875,0.600668,0.694309,0.128924,0.128924,0.330022,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.112216,3206
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,0.316219,0.270192,0.311790,0.233444,0.043938,None,0.578993,0.892754,0.623916,0.825595,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.107423,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286,2999
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,0.496960,0.313916,0.065828,0.238199,0.034472,None,0.491722,0.840941,0.913437,0.820266,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.204370,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000,3545


In [466]:
pred_column = [None]
for i in range(1, len(earnings)):
	if earnings[i] > earnings[i-1]:
		pred_column.append(1)
	else:
		pred_column.append(0)

In [467]:
pred_column

[None,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1]

In [468]:
final['pred_column'] = pred_column

In [469]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term,earnings,pred_column
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,0.463219,0.000000,0.340773,0.000000,1.000000,None,0.814983,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,0.216641,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,0.094461,0.000000,0.129221,0.192976,0.093333,0.108929,3034,NaN
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.402083,0.130598,0.612179,None,0.607133,0.341880,0.564629,0.924384,0.335772,0.435142,0.401507,0.377778,0.210688,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.129221,0.508306,0.016667,0.078571,3105,1.0
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.064420,0.183721,0.000000,None,0.568201,0.991736,0.762177,0.751299,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.216641,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.112216,3491,1.0
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.000000,0.170772,0.000150,None,0.278147,1.000000,0.870440,0.945994,0.645312,0.097877,0.097877,0.330022,0.210688,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.166832,0.112216,4011,1.0
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.043089,0.179941,0.039794,None,0.483256,0.867470,1.000000,0.933905,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.204370,0.227183,0.533160,0.483333,-0.009259,0.286658,0.129221,0.303163,0.189983,0.000000,3527,0.0
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,0.605006,0.433133,0.105704,0.209387,0.023048,None,0.780053,0.916031,0.677906,0.600114,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.204370,0.317708,0.136364,-0.254762,0.000000,0.286658,0.129221,0.447619,0.231664,0.112216,3133,0.0
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,0.231376,0.554699,0.094132,0.241033,0.021460,None,0.675224,0.830022,0.874324,0.766059,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.094461,0.286658,0.182041,0.300379,0.122743,0.051042,3191,1.0
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,1.000000,0.412158,0.122789,0.181647,0.060028,None,0.659204,0.828625,0.663875,0.600668,0.694309,0.128924,0.128924,0.330022,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.112216,3206,1.0
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,0.316219,0.270192,0.311790,0.233444,0.043938,None,0.578993,0.892754,0.623916,0.825595,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.107423,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286,2999,0.0
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,0.496960,0.313916,0.065828,0.238199,0.034472,None,0.491722,0.840941,0.913437,0.820266,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.204370,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000,3545,1.0


In [470]:
final.drop(['recommend_same'], axis=1, inplace=True)

In [471]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term,earnings,pred_column
0,Q1,0.569771,0.954499,0.965373,0.503842,1,2,2,0,0.148303,0.463219,0.000000,0.340773,0.000000,1.000000,0.814983,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,0.216641,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,0.094461,0.000000,0.129221,0.192976,0.093333,0.108929,3034,NaN
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.402083,0.130598,0.612179,0.607133,0.341880,0.564629,0.924384,0.335772,0.435142,0.401507,0.377778,0.210688,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.129221,0.508306,0.016667,0.078571,3105,1.0
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.064420,0.183721,0.000000,0.568201,0.991736,0.762177,0.751299,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.216641,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.112216,3491,1.0
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.000000,0.170772,0.000150,0.278147,1.000000,0.870440,0.945994,0.645312,0.097877,0.097877,0.330022,0.210688,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.166832,0.112216,4011,1.0
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.043089,0.179941,0.039794,0.483256,0.867470,1.000000,0.933905,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.204370,0.227183,0.533160,0.483333,-0.009259,0.286658,0.129221,0.303163,0.189983,0.000000,3527,0.0
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,0.605006,0.433133,0.105704,0.209387,0.023048,0.780053,0.916031,0.677906,0.600114,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.204370,0.317708,0.136364,-0.254762,0.000000,0.286658,0.129221,0.447619,0.231664,0.112216,3133,0.0
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,0.231376,0.554699,0.094132,0.241033,0.021460,0.675224,0.830022,0.874324,0.766059,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.094461,0.286658,0.182041,0.300379,0.122743,0.051042,3191,1.0
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,1.000000,0.412158,0.122789,0.181647,0.060028,0.659204,0.828625,0.663875,0.600668,0.694309,0.128924,0.128924,0.330022,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.112216,3206,1.0
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,0.316219,0.270192,0.311790,0.233444,0.043938,0.578993,0.892754,0.623916,0.825595,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.107423,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286,2999,0.0
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,0.496960,0.313916,0.065828,0.238199,0.034472,0.491722,0.840941,0.913437,0.820266,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.204370,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000,3545,1.0


In [474]:
final.dropna(inplace=True)

In [475]:
final.drop(['recommend_unknown', 'ceo_unknown', 'business_unknown'], axis=1, inplace=True)

In [476]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term,earnings,pred_column
1,Q2,0.461594,0.675262,0.942595,0.476743,1,2,2,0,0.270516,0.486264,0.103920,0.607133,0.341880,0.564629,0.924384,0.335772,0.435142,0.401507,0.377778,0.210688,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.129221,0.508306,0.016667,0.078571,3105,1.0
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.472980,0.780839,0.357279,0.568201,0.991736,0.762177,0.751299,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.216641,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.112216,3491,1.0
3,Q4,0.266704,0.827978,0.347529,0.373552,1,2,2,1,0.423630,0.947365,0.443284,0.278147,1.000000,0.870440,0.945994,0.645312,0.097877,0.097877,0.330022,0.210688,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.166832,0.112216,4011,1.0
4,Q5,0.588633,0.833791,0.780301,0.276677,1,2,2,2,0.204858,0.490547,0.211596,0.483256,0.867470,1.000000,0.933905,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.204370,0.227183,0.533160,0.483333,-0.009259,0.286658,0.129221,0.303163,0.189983,0.000000,3527,0.0
5,Q6,0.348993,0.895306,0.590667,0.359017,1,2,2,1,0.523169,0.605006,0.433133,0.780053,0.916031,0.677906,0.600114,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.204370,0.317708,0.136364,-0.254762,0.000000,0.286658,0.129221,0.447619,0.231664,0.112216,3133,0.0
6,Q7,0.633734,0.790989,0.773682,0.528625,1,2,2,2,0.293693,0.231376,0.554699,0.675224,0.830022,0.874324,0.766059,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.094461,0.286658,0.182041,0.300379,0.122743,0.051042,3191,1.0
7,Q8,0.322421,0.498108,0.629464,0.263246,1,2,2,2,0.517955,1.000000,0.412158,0.659204,0.828625,0.663875,0.600668,0.694309,0.128924,0.128924,0.330022,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.112216,3206,1.0
8,Q9,0.408722,0.964106,0.701467,0.294502,1,2,2,2,0.316600,0.316219,0.270192,0.578993,0.892754,0.623916,0.825595,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.107423,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286,2999,0.0
9,Q10,0.548412,1.000000,0.813475,0.214355,1,2,2,2,0.282548,0.496960,0.313916,0.491722,0.840941,0.913437,0.820266,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.204370,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000,3545,1.0
10,Q11,0.522389,0.857775,0.779167,0.204845,1,2,2,2,0.266945,0.634409,0.321658,0.475465,0.698413,0.656300,0.723669,0.823442,0.197691,0.166296,0.040762,0.127976,0.161714,0.365278,0.198710,0.087500,0.479877,0.024843,0.189394,0.800000,0.218552,0.061111,-0.050000,0.286658,0.129221,0.325442,0.290278,0.128571,3537,0.0


In [477]:
final.to_excel('ge_cleaned_data.xlsx')